In [135]:
import requests
from bs4 import BeautifulSoup
import os
import re
from collections import defaultdict
import json

In [5]:
response = requests.get('https://austinpollen.com/')

In [7]:
response.ok

True

In [35]:
response.text.split('\n')[37]

"         gtag('config', 'G-7BV2PLNGCY');"

In [16]:
soup = BeautifulSoup(response.text)

In [102]:
pattern = r'(?<=<script defer>)(.*)(?=</script>)'
script_text = re.findall(pattern, response.text, flags=re.DOTALL)[0]
script_text.split('\n')

['',
 "         google.charts.load('current', {'packages':['corechart','table','treemap','sankey']});",
 '   ',
 '         google.charts.setOnLoadCallback(drawChartindex);',
 '         google.charts.setOnLoadCallback(drawChartpollens);',
 '         google.charts.setOnLoadCallback(drawChartoakseason);',
 '         google.charts.setOnLoadCallback(drawChartelmseason);',
 '         google.charts.setOnLoadCallback(drawChartashseason);',
 '                                                                     ',
 '                                                                     ',
 '         window.onresize = function(event) { drawChartindex(); drawChartpollens(); drawChartoakseason(); drawChartelmseason(); drawChartashseason();} ',
 '',
 '',
 "   <!-- End section 'Top' -- Next section is 'DrawChart' -->",
 '   ',
 "   <!-- Begin section 'DrawChart index' -- Previous section was 'Top' -->",
 '',
 '         function drawChartindex() {',
 '         var data = google.visualization.arrayToData

In [113]:
test = defaultdict(int)
test['a'] += 1
test['b'] += 2
sum(test.values())

3

In [125]:
def find_json_text(text):
    started = False
    bracket_counts = defaultdict(int)
    for (i, c) in enumerate(text):
        if not started and c in '()[]{}':
            started = True
        if started:
            for bracket in [
                {'open':'(', 'close':')', 'name':'parenthesis'},
                {'open':'[', 'close':']', 'name':'parenthesis'},
                {'open':'{', 'close':'}', 'name':'parenthesis'},
            ]:
                if c == bracket['open']:
                    bracket_counts[bracket['name']] += 1
                if c == bracket['close']:
                    bracket_counts[bracket['name']] -= 1
                    if bracket_counts[bracket['name']] < 0:
                        raise Exception('Unexpected closing bracket')
            if sum(bracket_counts.values()) == 0:
                return text[:i]
    return ''
            

In [132]:
idx = script_text.find('function drawChartindex() {\n\s*var data = google.visualization.arrayToDataTable([ \n            ')
idx

-1

In [131]:
find_json_text(script_text[idx:])

'function drawChartindex('

In [144]:
script_text.find('function drawChartindex() {\n         var data = google.visualization.arrayToDataTable([ \n            ')
pattern = r'(?<=function drawChartindex\(\))(.*)(?<=arrayToDataTable\()(.*)'
text = find_json_text(re.findall(pattern, script_text, flags=re.DOTALL)[0][0]).split('\n')
text

[' {',
 '         var data = google.visualization.arrayToDataTable([ ',
 "            ['Factor', 'Average', {type: 'string', role: 'tooltip'}, {type: 'string', role: 'annotation'}, {'type': 'string', 'role': 'style'}],",
 "            ['Indoor Dust, Dander', 700, 'Indoor Dust, Dander\\n\\nUnhealthy for Sensitive Groups ~ 6 out of 12  (Thursday 4/21)\\n\\nFrom 1 source:\\n-Austin AccuWeather: 6 out of 12 (4/21@7:00am)\\n', '(1)', 'bar {size: 5 ; stroke-color: #666666; fill-color: #C16937 }'],",
 "            ['Cladosporium', 549, 'Cladosporium (a genus of fungi)\\n\\nHigh ~ 20217 gr/m\\u00B3  (Thursday 4/21)\\n\\nFrom one source:\\n-SA West AAAAI: 20217 gr/m\\u00B3 (4/21@1:31pm)\\n', '(1)', 'bar {size: 5 ; stroke-color: #666666; fill-color: #D6A232 }'],",
 "            ['Asthma Index', 535, 'Asthma Index (a composite index)\\n\\nHigh ~ 6 out of 12 (Thursday 4/21)\\n\\nWeighted average of 2 sources:\\n-Austin AccuWeather: 4 out of 12 (4/21@7:00am)\\n-Austin AsthmaForecast.com: 8 out of 1

In [168]:
test = text[3]
test
first_quote_idx = test.find("'")
test[first_quote_idx+1:].find("'")
test.find('fill-color')
pattern = '#[0-9A-F]*'
re.search(pattern, test[test.find('fill-color'):]).group(0)


'#C16937'

In [170]:
def parse_factor(text):
    first_quote_idx = text.find("'")
    second_quote_idx = text[first_quote_idx+1:].find("'") + first_quote_idx + 1
    factor = text[first_quote_idx+1:second_quote_idx]
    value = text[second_quote_idx+1:].split(',')[1]
    if re.search('#[0-9A-F]*', text[text.find('fill-color'):]):
        fill_color = re.search('#[0-9A-F]*', text[text.find('fill-color'):]).group(0)
    else:
        fill_color = ''
    return factor, value, fill_color

[parse_factor(x) for x in text if x.startswith('            [') and parse_factor(x)[0] != 'Factor']

[('Indoor Dust, Dander', ' 700', '#C16937'),
 ('Cladosporium', ' 549', '#D6A232'),
 ('Asthma Index', ' 535', '#D6A232'),
 ('Oak', ' 515', '#D6A232'),
 ('Trees', ' 468', '#9AA22B'),
 ('Graminales', ' 400', '#9AA22B'),
 ('Molds', ' 351', '#9AA22B'),
 ('Grasses', ' 282', '#9AA22B'),
 ('Cedar', ' 275', '#9AA22B'),
 ('Hickory/Pecan', ' 236', '#006A92')]

In [53]:
response.text.split('\n')

['<html>',
 '   <head>',
 "   <!-- Begin section 'Top (multi)' Also works for singles -->",
 '',
 '      <meta name="viewport" content="width=device-width, initial-scale=0.86, maximum-scale=5.0, minimum-scale=0.86">',
 '',
 '      <title>Austin Pollen | Austin pollen count allergy cedar mold ragweed oak</title>',
 '      <meta name="description" content="Austin pollen count and report. Allergy count report and help. Mold count report allergy. Best source daily allergy report. Cedar Oak Ragweed Mold.">',
 '      <meta name="author" content="Austin Pollen">',
 '',
 '      <meta property="og:title" content="Austin Pollen | Austin pollen count allergy cedar mold ragweed oak" />',
 '      <meta property="og:description" content="Austin pollen count and report. Allergy count report and help. Mold count report allergy. Best source daily allergy report. Cedar Oak Ragweed Mold." />',
 '      <meta property="og:url" content="https://austinpollen.com/index.html?version=1650584211" />',
 '',
 '   

In [31]:
soup.findAll('script', {'defer':''})

[<script src="https://www.gstatic.com/charts/loader.js"></script>,
 <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-7BV2PLNGCY"></script>,
 <script>
          window.dataLayer = window.dataLayer || [];
          function gtag(){dataLayer.push(arguments);}
          gtag('js', new Date());
          gtag('config', 'G-7BV2PLNGCY');
       </script>,
 <script defer="">
          google.charts.load('current', {'packages':['corechart','table','treemap','sankey']});
    
          google.charts.setOnLoadCallback(drawChartindex);
          google.charts.setOnLoadCallback(drawChartpollens);
          google.charts.setOnLoadCallback(drawChartoakseason);
          google.charts.setOnLoadCallback(drawChartelmseason);
          google.charts.setOnLoadCallback(drawChartashseason);
                                                                      
                                                                      
          window.onresize = function(event) { drawChartinde

In [14]:
os.stat('news.json')

os.stat_result(st_mode=33206, st_ino=16888498602765335, st_dev=575024205, st_nlink=1, st_uid=0, st_gid=0, st_size=16154, st_atime=1650603599, st_mtime=1650602457, st_ctime=1650602424)

In [15]:
os.stat('asdfasdf')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'asdfasdf'

In [173]:
os.path.getmtime('pollen.json')

1650644877.4000113

In [174]:
translate = {
    'abc':'def',
    'a':'d'
}

words = ['a', 'b', 'c', 'abc']
[translate[w] if w in translate else w for w in words]

['d', 'b', 'c', 'def']